In [88]:
import tensorflow as tf
import cv2
import numpy as np
import os

x = tf.placeholder(tf.float32,[None, 64, 64, 3])
y = tf.placeholder(tf.float32, [None])
phase = tf.placeholder(tf.bool, [1], name='phase')
keep_prob = tf.placeholder(tf.float32)
batch_size = 64

with open('/home/cqm/datasets/celeba/index.txt') as input_file:
    person_index = []
    for line in input_file.readlines():
        line = line.strip('\n').split(" ")
        person_index.append(line[1:])
path = '/home/cqm/datasets/celeba/img/'  

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.01)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

def conv2d(x, w):
    return tf.nn.conv2d(x, w, strides = [1, 1, 1, 1], padding = "SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")

mybn = tf.contrib.layers.batch_norm
def network(x):
    w_conv1 = weight_variable([5, 5, 3, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(mybn(conv2d(x, w_conv1) + b_conv1, center=True, scale=True, is_training=phase[0]))
    h_pool1 = max_pool_2x2(h_conv1) #32*32*32

    w_conv2 = weight_variable([3, 3, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(mybn(conv2d(h_pool1, w_conv2) + b_conv2, center=True, scale=True, is_training=phase[0]))
#     h_pool2 = max_pool_2x2(h_conv2) #64*16*16
    w_conv2_2 = weight_variable([3, 3, 64, 64])
    b_conv2_2 = bias_variable([64])
    h_conv2 = tf.nn.relu(mybn(conv2d(h_conv2, w_conv2_2) + b_conv2_2, center=True, scale=True, is_training=phase[0]))
    h_pool2 = max_pool_2x2(h_conv2) #64*16*16

    w_conv3 = weight_variable([3, 3, 64, 128])
    b_conv3 = bias_variable([128])
    h_conv3 = tf.nn.relu(mybn(conv2d(h_pool2, w_conv3) + b_conv3, center=True, scale=True, is_training=phase[0]))
#     h_pool3 = max_pool_2x2(h_conv3) #64*8*8
    w_conv3_2 = weight_variable([3, 3, 128, 128])
    b_conv3_2 = bias_variable([128])
    h_conv3 = tf.nn.relu(mybn(conv2d(h_conv3, w_conv3_2) + b_conv3_2, center=True, scale=True, is_training=phase[0]))
    h_pool3 = max_pool_2x2(h_conv3) #64*8*8
    
    w_conv4 = weight_variable([3, 3, 128, 256])
    b_conv4 = bias_variable([256])
    h_conv4 = tf.nn.relu(mybn(conv2d(h_pool3, w_conv4) + b_conv4, center=True, scale=True, is_training=phase[0]))
#     h_pool4 = max_pool_2x2(h_conv4) #128*4*4
    w_conv4_2 = weight_variable([3, 3, 256, 256])
    b_conv4_2 = bias_variable([256])
    h_conv4 = tf.nn.relu(mybn(conv2d(h_conv4, w_conv4_2) + b_conv4_2, center=True, scale=True, is_training=phase[0]))
    h_pool4 = max_pool_2x2(h_conv4) #128*4*4
    
    h_pool4_reshape = tf.reshape(h_pool4, [-1, 4*4*256])

    w_fc1 = weight_variable([4*4*256, 512])
    b_fc1 = bias_variable([512])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool4_reshape, w_fc1) + b_fc1)
    
    w_fc2 = weight_variable([512, 512])
    h_fc1 = tf.matmul(h_fc1, w_fc2)
    #h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    #w_fc2 = weight_variable([512, classnum])
    #b_fc2 = bias_variable([classnum])
    #y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) +b_fc2)
    #y_conv = tf.add(tf.matmul(h_fc1, w_fc2) +b_fc2)
    return h_fc1

def contrastive_loss(y, d):
    tmp = y*tf.square(d)
    tmp2 = (1-y)*tf.square(tf.maximum((20-d), 0))
    return tf.reduce_sum(tmp+tmp2)/batch_size

out = network(x)
    
d = tf.reduce_sum((out[:batch_size] - out[batch_size:]+1e-5)**2, axis = 1)**0.5
loss = contrastive_loss(y, d)
train_step = tf.train.AdamOptimizer(2e-5).minimize(loss)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

with open('/home/cqm/datasets/celeba/index.txt') as input_file:
    person_index = []
    for line in input_file.readlines():
        line = line.strip('\n').split(" ")
        person_index.append(line[1:])

for i in range(5000):
    pos_pair_list1 = []
    pos_pair_list2 = []
    while len(pos_pair_list1) < 32:
        id = np.random.choice(len(person_index),1,replace=False)[0]
        if len(person_index[id]) >= 2:
            pair_id = np.random.choice(len(person_index[id]),2,replace=False)
            pos_pair_list1.append(person_index[id][pair_id[0]])
            pos_pair_list2.append(person_index[id][pair_id[1]])

    neg_pair_list1 = []
    neg_pair_list2 = []
    while len(neg_pair_list1) < 32:
        id = np.random.choice(len(person_index),2,replace=False)
        neg_id1 = np.random.choice(len(person_index[id[0]]),1,replace=False)[0]
        neg_id2 = np.random.choice(len(person_index[id[1]]),1,replace=False)[0]
        neg_pair_list1.append(person_index[id[0]][neg_id1])
        neg_pair_list2.append(person_index[id[1]][neg_id2])
    
    x_imgs = []
#     y_labels = [1 for num in range(32)]+[0 for num in range(32)]
    y_labels = [1] * 32 + [0] * 32
    for img_name in pos_pair_list1:
        img = cv2.imread(path + img_name)
        img = cv2.resize(img, (64, 64))/127.5 - 1.0
        x_imgs.append(img)
    for img_name in neg_pair_list1:
        img = cv2.imread(path + img_name)
        img = cv2.resize(img, (64, 64))/127.5 - 1.0
        x_imgs.append(img)
    for img_name in pos_pair_list2:
        img = cv2.imread(path + img_name)
        img = cv2.resize(img, (64, 64))/127.5 - 1.0
        x_imgs.append(img)
    for img_name in neg_pair_list2:
        img = cv2.imread(path + img_name)
        img = cv2.resize(img, (64, 64))/127.5 - 1.0
        x_imgs.append(img)

    #print y_labels
    x_imgs = np.array(x_imgs, dtype=np.float32)

    if i%10 == 0:
        train_loss = loss.eval(feed_dict={x:x_imgs,y:y_labels,phase:[True]})
        print('step %d , training loss %f'%(i, train_loss))
    train_step.run(feed_dict={x:x_imgs,y:y_labels,phase:[True]})

step 0 , training loss 168.181427
step 10 , training loss 138.362961
step 20 , training loss 154.492706
step 30 , training loss 120.378342
step 40 , training loss 111.209435
step 50 , training loss 91.913033
step 60 , training loss 94.863815
step 70 , training loss 107.805054
step 80 , training loss 107.882843
step 90 , training loss 84.134720
step 100 , training loss 102.999771
step 110 , training loss 101.733337
step 120 , training loss 72.625153
step 130 , training loss 90.436523
step 140 , training loss 91.049835
step 150 , training loss 107.345474
step 160 , training loss 95.233780
step 170 , training loss 77.854141
step 180 , training loss 102.719269
step 190 , training loss 78.661095
step 200 , training loss 84.749496
step 210 , training loss 83.661148
step 220 , training loss 80.462875
step 230 , training loss 90.011787
step 240 , training loss 99.497734
step 250 , training loss 102.271225
step 260 , training loss 86.126190
step 270 , training loss 78.429474
step 280 , training

step 2310 , training loss 58.640518
step 2320 , training loss 65.769264
step 2330 , training loss 52.335022
step 2340 , training loss 56.938129
step 2350 , training loss 53.963493
step 2360 , training loss 60.343254
step 2370 , training loss 54.939705
step 2380 , training loss 55.681995
step 2390 , training loss 44.607243
step 2400 , training loss 75.321793
step 2410 , training loss 58.956978
step 2420 , training loss 58.449211
step 2430 , training loss 55.178696
step 2440 , training loss 54.317665
step 2450 , training loss 56.763100
step 2460 , training loss 38.657558
step 2470 , training loss 49.722961
step 2480 , training loss 56.467300
step 2490 , training loss 62.934036
step 2500 , training loss 61.692169
step 2510 , training loss 60.657623
step 2520 , training loss 64.059448
step 2530 , training loss 43.382584
step 2540 , training loss 55.755833
step 2550 , training loss 55.517200
step 2560 , training loss 77.339554
step 2570 , training loss 58.337132
step 2580 , training loss 55

step 4590 , training loss 58.129784
step 4600 , training loss 53.641533
step 4610 , training loss 39.832153
step 4620 , training loss 60.039154
step 4630 , training loss 65.603867
step 4640 , training loss 55.900826
step 4650 , training loss 39.694130
step 4660 , training loss 38.759819
step 4670 , training loss 52.268997
step 4680 , training loss 59.236492
step 4690 , training loss 46.182652
step 4700 , training loss 40.171631
step 4710 , training loss 58.582047
step 4720 , training loss 63.444298
step 4730 , training loss 53.349178
step 4740 , training loss 52.686203
step 4750 , training loss 51.612736
step 4760 , training loss 40.926849
step 4770 , training loss 59.390347
step 4780 , training loss 48.495026
step 4790 , training loss 49.616570
step 4800 , training loss 54.780056
step 4810 , training loss 59.636848
step 4820 , training loss 51.537842
step 4830 , training loss 49.891907
step 4840 , training loss 53.925339
step 4850 , training loss 47.923439
step 4860 , training loss 45

In [89]:
test1 = cv2.imread('1/103.jpg')
test2 = cv2.imread('1/104.jpg')
test3 = cv2.imread('2/203.jpg')
test4 = cv2.imread('2/204.jpg')

In [90]:
test1 = test1.astype('float32')[None]/127.5 - 1.0
test2 = test2.astype('float32')[None]/127.5 - 1.0
test3 = test3.astype('float32')[None]/127.5 - 1.0
test4 = test4.astype('float32')[None]/127.5 - 1.0

In [91]:
res1 = out.eval(feed_dict={x:test1,phase:[False]})
res2 = out.eval(feed_dict={x:test2,phase:[False]})
res3 = out.eval(feed_dict={x:test3,phase:[False]})
res4 = out.eval(feed_dict={x:test4,phase:[False]})

In [92]:
print(((res3 - res4)**2).sum())
print(((res2 - res4)**2).sum())
print(((res2 - res3)**2).sum())
print(((res1 - res4)**2).sum())
print(((res1 - res3)**2).sum())
print(((res1 - res2)**2).sum())

1.09873e-08
4.2052855e-09
7.4234423e-09
9.656846e-09
3.8135582e-08
1.8269589e-08


In [93]:
person1 = []
for name in sorted(os.listdir('test_faces/1'))[1:]:
    img = cv2.imread('test_faces/1/' + name)
    img = cv2.resize(img, (64, 64))/127.5 - 1.0
    person1.append(img)
person1 = np.array(person1, dtype=np.float32)
feature1 = out.eval(feed_dict={x:person1,phase:[False]})

person2 = []
for name in sorted(os.listdir('test_faces/2'))[1:]:
    img = cv2.imread('test_faces/2/' + name)
    img = cv2.resize(img, (64, 64))/127.5 - 1.0
    person2.append(img)
person2 = np.array(person2, dtype=np.float32)
feature2 = out.eval(feed_dict={x:person2,phase:[False]})

In [94]:
def calc_dist(x,y):
    return np.sum((x[None] - y[:,None])**2, axis=2)

In [95]:
calc_dist(feature1,feature1).mean()

2.0784698e-08

In [96]:
calc_dist(feature2,feature2).mean()

1.8749807e-08

In [97]:
calc_dist(feature1,feature2).mean()

2.0036566e-08

In [98]:
saver = tf.train.Saver(max_to_keep = 1)
saver.save(sess,'face_rec.ckpt')

'face_rec.ckpt'

In [85]:
print res1

[[ 1.81823998e-05 -1.07714068e-03  1.03330925e-01 -6.87391907e-02
  -8.72975588e-02  4.42221090e-02  1.09272506e-02  1.11805275e-01
   2.59255506e-02  4.56141382e-02  7.12151080e-02  2.18364239e-01
   3.54588591e-02  3.51760685e-02 -1.47523180e-01 -4.73085493e-02
  -5.71580231e-03 -2.14989334e-01 -2.83095576e-02 -9.61689427e-02
  -1.03918821e-01  3.13162319e-02  1.12282112e-01  1.38177136e-02
   8.87697339e-02 -8.75265151e-02  3.59175242e-02 -6.44234046e-02
   8.70271865e-03  9.75348875e-02  7.94809610e-02  1.60685807e-01
   4.02285345e-02  4.79413122e-02 -1.19370505e-01 -6.82168528e-02
   1.20263509e-01 -1.48274601e-01 -6.30757660e-02 -7.11271819e-03
   5.57034984e-02  6.97342157e-02 -4.19762991e-02  3.91061455e-02
  -1.52945099e-02 -1.05433613e-02 -5.31602129e-02 -4.03903201e-02
  -2.83444561e-02  4.51139323e-02 -3.34570371e-02 -1.82932705e-01
  -1.14349656e-01 -8.99200095e-05  3.22412979e-03  6.26255646e-02
  -3.46180499e-02  1.72794744e-01  1.21809039e-02 -2.28622228e-01
  -5.41535